In [1]:
with open('区域位置信息.txt','r') as f:
    string = f.read()

list = eval(string)
print(len(list))


1024


In [2]:
import csv

csv_reader = csv.reader(open('上海区域交通信息.csv','r'))
data = []
for item in csv_reader:
    data.append(item)

print(len(data))


2479


In [3]:
import pandas as pd

res = pd.read_csv('上海区域交通信息.csv')
res.head()

,time,r1,r2,r3,r4,r5,r6,r7,r8,r9,...,r1015,r1016,r1017,r1018,r1019,r1020,r1021,r1022,r1023,r1024
0,2020_10_29 13:39:03,0.200,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0
1,2020_10_29 13:40:00,0.200,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020_10_29 14:00:00,0.333,0.5,0.5,0.5,0.5,0.5,0.5,0.167,0.167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020_10_29 14:20:00,0.000,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020_10_29 14:40:00,0.200,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
import numpy as np
info = np.array(res.iloc[:,1:])
print(info)



[[ 0.2    0.    -1.    ...  0.2    0.     0.   ]
 [ 0.2    0.    -1.    ...  0.     0.     0.   ]
 [ 0.333  0.5    0.5   ...  0.     0.     0.   ]
 ...
 [ 0.     0.    -1.    ...  0.     0.     0.   ]
 [ 0.4    0.667 -1.    ...  0.143  0.333  0.333]
 [ 0.4    0.667 -1.    ...  0.286  0.167  0.333]]


In [5]:
def drop_none(info):
    rows,cols=info.shape
    info_cpy = info.copy()

    for j in range(0,cols):
        num = 0
        sum = 0
        for i in range(0,rows):
            if info[i][j]>=0:
                sum = sum + info[i][j]
                num = num + 1
        avg = sum/num

        if num < (rows/2):
            info_cpy[:,j] = 0
        else:
            for i in range(0,rows):
                if info[i][j]<0:
                    info_cpy[i][j] = round(avg,3)
    
    return info_cpy

info_cpy = drop_none(info)
print(info_cpy)


[[0.2   0.    0.    ... 0.2   0.    0.   ]
 [0.2   0.    0.    ... 0.    0.    0.   ]
 [0.333 0.5   0.    ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.4   0.667 0.    ... 0.143 0.333 0.333]
 [0.4   0.667 0.    ... 0.286 0.167 0.333]]


In [6]:
info_int = np.rint(info_cpy*10)
print(info_int)

[[2. 0. 0. ... 2. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 [3. 5. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [4. 7. 0. ... 1. 3. 3.]
 [4. 7. 0. ... 3. 2. 3.]]


In [7]:
def random_entropy(info):
    re = np.zeros(info.shape[1])
    len = info.shape[1]
    for i in np.arange(0, len):
        unique = np.unique(info[:,i])
        N = unique.shape[0]
        re[i] = np.log2(N)
    
    return re

re = random_entropy(info_int)
print(re)
    
        

[3.         3.169925   0.         ... 2.80735492 2.5849625  2.80735492]


In [8]:
print(info_int.shape)
print(np.log2(np.unique(info_int[:,info_int.shape[1]-1]).shape[0]))

(2478, 1024)
2.807354922057604


In [9]:
def shannon_entropy(info):
    se = np.zeros(info.shape[1])
    len = info.shape[1]
    size = info.shape[0]
    # 计算每一列
    for i in np.arange(0,len):
        unique = np.unique(info[:,i])
        tmp = 0
        # 计算每一列中每个数值出现的概率
        for j in np.arange(0, unique.shape[0]):
            num = np.sum(info_int[:,i]==unique[j])
            p = num/size
            tmp += -(p*np.log2(p))

        se[i] = tmp

    return se

se = shannon_entropy(info_int)
print(se)





[2.14038312 2.0912635  0.         ... 1.61279104 1.12465446 1.14221731]


In [10]:
def real_entropy(info):
    re = np.zeros(info.shape[1])
    len1 = info.shape[1]
    size = info.shape[0]
    # 计算每一列
    for i in np.arange(0,len1):
        unique = np.unique(info[:,i])
        stlist = [ str(int(k)) for k in unique]

        p = ""
        for c in info[:,i]:
            pc = p + str(int(c))
            if pc in stlist:
                p = pc
            else:
                stlist.append(pc)
                p = str(int(c))
        
        sum = 0
        for st in stlist:
            sum += len(st)
        
        re[i] = (size/sum)*np.log(size)
    return re

real = real_entropy(info_int)
print(real)

[6.08996324 6.06706864 7.79319239 ... 6.32671777 6.75482494 6.73134623]


In [17]:
def findx(info, e, n):
    # e: 代表熵， 是个数组；n: 代表迭代轮数
    row = info.shape[0]
    col = info.shape[1]
    res = np.zeros(col)

    # 对于每一列来说：
    for i in np.arange(0, col):
        x = 0.1
        N = np.unique(info[:,i]).shape[0]
        S = e[i]
        # 计算x近似值
        if N>1:
            for j in np.arange(0, n):
                fx = -x*np.log2(x)-(1-x)*np.log2(1-x)+(1-x)*np.log2(N-1)-S
                fx2 = np.log2(1-x)-np.log2(x)-np.log2(N-1)
                if fx2!=0:
                    x = x-fx/fx2
        res[i] = x
    return res

x = findx(info_int, re, 100)
print(x)


[0.125      0.11111111 0.1        ... 0.14285714 0.16666667 0.14285714]


In [31]:
def aa(info, e, x, mask):
    # e: 代表熵， 是个数组；n: 代表迭代轮数
    row = info.shape[0]
    col = info.shape[1]
    res = np.zeros(col)

    # 对于每一列来说：
    for i in np.arange(0, col):
        if mask[i]==1:
            x = 0.1
            N = np.unique(info[:,i]).shape[0]
            S = e[i]
            fx = -x*np.log2(x)-(1-x)*np.log2(1-x)+(1-x)*np.log2(N-1)-S
            
            if abs(fx)<0.001:
                print(i,":",fx)

aa(info_int, re, x, mask)


1 : -0.0009294078530306393
8 : -0.0009294078530306393
22 : -0.0009294078530306393
25 : -0.0009294078530306393
32 : -0.0009294078530306393
34 : -0.0009294078530306393
38 : -0.0009294078530306393
40 : -0.0009294078530306393
57 : 0.0
58 : -0.0009294078530306393
59 : -0.0009294078530306393
65 : -0.0009294078530306393
68 : -0.0009294078530306393
70 : -0.0009294078530306393
71 : -0.0009294078530306393
73 : -0.0009294078530306393
74 : 0.0
75 : -0.000700739649389881
85 : 0.0
89 : -0.0009294078530306393
91 : -0.0009294078530306393
100 : -0.0009294078530306393
110 : -0.0009294078530306393
119 : -0.0009294078530306393
129 : -0.0009294078530306393
130 : -0.0009294078530306393
139 : -0.0009294078530306393
151 : -0.0009294078530306393
152 : -0.0009294078530306393
169 : -0.0009294078530306393
184 : -0.0009294078530306393
185 : 0.0
187 : -0.0009294078530306393
188 : -0.0009294078530306393
193 : -0.0009294078530306393
194 : 0.0
214 : -0.0009294078530306393
215 : -0.0009294078530306393
217 : -0.00092940

In [28]:
def get_mask(info):
    row = info.shape[0]
    col = info.shape[1]

    mask = np.zeros(col)
    for i in np.arange(0, col):
        bg0 = np.sum(info[:,i]>=0)
        if bg0 >= info.shape[0]*0.8:
            mask[i] = 1
        else:
            mask[i] = 0
    return mask

mask = get_mask(info)